In [1]:
import $ivy.`org.typelevel::cats-core:2.9.0`

import cats._
import cats.syntax.all._
import cats.data.Nested

extension [A](a: A) {
  inline def |>[B](inline f: A => B): B = f(a)
}

import $ivy.$                               


import cats._

import cats.syntax.all._

import cats.data.Nested


defined extension methods 

In [2]:
// "Schema"
class V()
class E(val src: V, val tgt: V)
class G(val vs: List[V], val es: List[E])

defined class V
defined class E
defined class G

In [3]:
// "Outside-In" Model
object OM:
    // Edge "Attrs"
    def src(e: E): V = e.src 
    def tgt(e: E): V = e.tgt

    // Graph "Attrs"
    def vs(g: G): List[V] = g.vs 
    def es(g: G): List[E] = g.es

    // Bundles 
    def outgoing(v: V)(using g: G): List[E] = g.es.filter(src(_) == v)
    def incoming(v: V)(using g: G): List[E] = g.es.filter(tgt(_) == v)

    // Spans
    def oneighbors(v: V)(using g: G): List[V] = g.vs.filter(outgoing(v).map(tgt).contains(_))
    def ineighbors(v: V)(using g: G): List[V] = g.vs.filter(incoming(v).map(src).contains(_))
    def neighbors(v: V)(using g: G): List[V] = oneighbors(v) ++ ineighbors(v)

    // Generic
    def sum[A](using num: Numeric[A])(as: List[A]): A = as.sum
    def size[A](as: List[A]): Int = as.size
    def range(i: Int): List[Int] = List.range(0, i)

defined object OM

In [4]:
// Keeps nesting unchanged
def fmap[A, B, F[_]](f: A => B)(using F: Functor[F]): (F[A]) => F[B] = 
    xs => xs map f

defined function fmap

In [5]:
// Produces a layer of nesting
def pmap[A, B, F[_], G[_]](f: A => G[B])(using F: Functor[F]): (F[A]) => Nested[F, G, B] = 
    xs => Nested(xs map f)

defined function pmap

In [6]:
// Reduces a layer of nesting
def rmap[A, B, F[_], G[_]](f: G[A] => B)(using F: Functor[F]): (Nested[F, G, A]) => F[B] = 
    xs => xs.value map f

defined function rmap

In [7]:
// "Inside-Out" Model
object IM:
    // Edge "Attrs"
    def src[F[_]](fe: F[E])(using F: Functor[F]): F[V] = fmap(OM.src)(fe)
    def tgt[F[_]](fe: F[E])(using F: Functor[F]): F[V] = fmap(OM.tgt)(fe)

    // Graph "Attrs"
    def vs[F[_]](fg: F[G])(using F: Functor[F]): Nested[F, List, V] = pmap(OM.vs)(fg)
    def es[F[_]](fg: F[G])(using F: Functor[F]): Nested[F, List, E] = pmap(OM.es)(fg)

    // Bundles
    def outgoing[F[_]](fv: F[V])(using F: Functor[F])(using g: G): Nested[F, List, E] = pmap(OM.outgoing)(fv)
    def incoming[F[_]](fv: F[V])(using F: Functor[F])(using g: G): Nested[F, List, E] = pmap(OM.incoming)(fv)

    // Spans
    def oneighbors[F[_]](fv: F[V])(using F: Functor[F])(using g: G): Nested[F, List, V] = pmap(OM.oneighbors)(fv)
    def ineighbors[F[_]](fv: F[V])(using F: Functor[F])(using g: G): Nested[F, List, V] = pmap(OM.ineighbors)(fv)
    def neighbors[F[_]](fv: F[V])(using F: Functor[F])(using g: G): Nested[F, List, V] = pmap(OM.neighbors)(fv)

    // Generic
    def sum[A, F[_]](fa: Nested[F, List, A])(using F: Functor[F])(using num: Numeric[A]): F[A] = rmap(OM.sum)(fa)
    def size[A, F[_]](fa: Nested[F, List, A])(using F: Functor[F]): F[Int] = rmap[A,Int,F,List](OM.size)(fa)
    def range[F[_]](fi: F[Int])(using F: Functor[F]): Nested[F, List, Int] = pmap(OM.range)(fi)

defined object IM

In [8]:
// "Instance"

val v0 = V()
val v1 = V()
val v2 = V()
val e0 = E(v0, v1)
val e1 = E(v1, v2)

val g = G(List(v0,v1,v2), List(e0,e1))

given global: G = g

v0: V = ammonite.$sess.cmd1$Helper$V@584ccde4
v1: V = ammonite.$sess.cmd1$Helper$V@4140e3cc
v2: V = ammonite.$sess.cmd1$Helper$V@5a66922f
e0: E = ammonite.$sess.cmd1$Helper$E@71934025
e1: E = ammonite.$sess.cmd1$Helper$E@2f35bab1
g: G = ammonite.$sess.cmd1$Helper$G@2f7059c3
global: G = <given>

## "Inside-Out" Examples
Nesting is implicitly managed by the type system.

In [9]:
val fg = List(g)

fg: List[G] = List(ammonite.$sess.cmd1$Helper$G@2f7059c3)

In [10]:
fg 
    |> IM.vs         // Produce "List"

res9: Nested[List, List, V] = Nested(
  value = List(
    List(
      ammonite.$sess.cmd1$Helper$V@584ccde4,
      ammonite.$sess.cmd1$Helper$V@4140e3cc,
      ammonite.$sess.cmd1$Helper$V@5a66922f
    )
  )
)

In [11]:
fg 
    |> IM.vs         // Produce "List"
    |> IM.neighbors  // Produce "List"

res10: Nested[_root_.cats.data.Nested[[A >: scala.Nothing <: scala.Any] => _root_.cats.data.Nested[scala.collection.immutable.List, scala.List, A], scala.List, ammonite.$sess.cmd6.wrapper.cmd1.V], List, V] = Nested(
  value = Nested(
    value = List(
      List(
        List(ammonite.$sess.cmd1$Helper$V@4140e3cc),
        List(
          ammonite.$sess.cmd1$Helper$V@5a66922f,
          ammonite.$sess.cmd1$Helper$V@584ccde4
        ),
        List(ammonite.$sess.cmd1$Helper$V@4140e3cc)
      )
    )
  )
)

In [12]:
fg 
    |> IM.vs         // Produce "List"
    |> IM.neighbors  // Produce "List"
    |> IM.size       // Reduce "List"

res11: Nested[List, List, Int] = Nested(value = List(List(1, 2, 1)))

In [13]:
fg 
    |> IM.vs         // Produce "List"
    |> IM.neighbors  // Produce "List"
    |> IM.size       // Reduce "List"
    |> IM.range      // Produce "List"
    |> IM.sum        // Reduce "List"

res12: Nested[List, List, Int] = Nested(value = List(List(0, 1, 0)))

## "Outside-In" Examples

Nesting is explicitly managed by the user.

In [14]:
val fg = List(g)

fg: List[G] = List(ammonite.$sess.cmd1$Helper$G@2f7059c3)

In [15]:
fg 
    |> fmap(OM.vs)

res14: List[List[V]] = List(
  List(
    ammonite.$sess.cmd1$Helper$V@584ccde4,
    ammonite.$sess.cmd1$Helper$V@4140e3cc,
    ammonite.$sess.cmd1$Helper$V@5a66922f
  )
)

In [16]:
fg 
    |> fmap(OM.vs)
    |> fmap(fmap(OM.neighbors))

res15: List[List[List[V]]] = List(
  List(
    List(ammonite.$sess.cmd1$Helper$V@4140e3cc),
    List(
      ammonite.$sess.cmd1$Helper$V@5a66922f,
      ammonite.$sess.cmd1$Helper$V@584ccde4
    ),
    List(ammonite.$sess.cmd1$Helper$V@4140e3cc)
  )
)

In [17]:
fg 
    |> fmap(OM.vs)
    |> fmap(fmap(OM.neighbors))
    |> fmap(fmap(OM.size))

res16: List[List[Int]] = List(List(1, 2, 1))

In [18]:
fg 
    |> fmap(OM.vs)
    |> fmap(fmap(OM.neighbors))
    |> fmap(fmap(OM.size))
    |> fmap(fmap(OM.range))
    |> fmap(fmap(OM.sum))

res17: List[List[Int]] = List(List(0, 1, 0))